In [8]:
from starter import *
from anneal import *
import math
import multiprocess as mp
import copy, random
from collections import defaultdict, Counter
import itertools
from bs4 import BeautifulSoup
import requests
from heapq import heapify, heappop, heappush
tar('outputs', overwrite = True)

In [2]:
tar('outputs', overwrite = True)

In [3]:
def apply_pi(G, pi):
    num_v = len(G.nodes)
    assert num_v == len(pi)
    max_p = max(pi)

    p = list(Counter(pi))
    p.sort()
    map_p = {p[i]:i for i in range(len(p))}
    for v in range(num_v):
        G.nodes[v]['team'] = int(map_p[pi[v]] + 1)
    return G  

In [4]:
def get_targets(file):
    f = open(file, 'r')
    lines = f.readlines()
    targets = []
    for line in lines:
        name, num, k = line.strip().split()
        targets.append((name, float(num), int(k)))
    return targets
targets = get_targets('target_soln_new.txt')

In [5]:
def write(instance, in_file, out_file, file_name, cost, k, overwrite):
    df = pd.read_csv('best_scores.csv')
    r = df.index[df['name'] == file_name]
    original = float(df.loc[r, 'score'])
    if cost < original:
        df.loc[r, 'score'] = cost
        df.loc[r, 'k'] = k
        df.to_csv('best_scores.csv', index=False)
        print(f"{str(in_file)}: cost", original, '->',score(instance))
        write_output(instance, out_file, overwrite)

In [7]:
df = pd.read_csv('best_scores.csv')
df2 = pd.read_csv('best_scores_first.csv')
for file, cost_my, cost_target in targets:
    G = read_input('inputs/'+file)
    f_my = open(str(Path('outputs') / f"{file[:-len('.in')]}.out"))
    f_first = open(str(Path('outputs_first') / f"{file[:-len('.in')]}.out"))
    pi_my = json.load(f_my)
    apply_pi(G, pi_my)
    score_my = score(G)
    
    pi_first = json.load(f_first)
    apply_pi(G, pi_first)
    score_first = score(G)
    
    r = df.index[df['name'] == file]
    k = int(df.loc[r, 'k'])
    
    if score_first < score_my:
        write(G,'inputs/'+file,str(Path('outputs') / f"{file[:-len('.in')]}.out"), file, score_first,k, True)
    f_my.close()
    f_first.close()

inputs/medium224.in: cost 34540.042697073266 -> 34365.70364765726
inputs/medium126.in: cost 2009.5536923187667 -> 1813.905609893065
inputs/large224.in: cost 38038.45954827273 -> 37926.69107937125
inputs/small34.in: cost 169821.7311602446 -> 166319.97388281234
inputs/small102.in: cost 130122.14243574018 -> 129067.90918776828
inputs/medium151.in: cost 35974.870601390765 -> 31503.522451794142
inputs/small118.in: cost 160475.41592657613 -> 159953.49323151432
inputs/small251.in: cost 23796.992492811816 -> 23341.99945932661
inputs/small141.in: cost 4173.125063845673 -> 3894.283972521378


In [11]:
### Naive Partitioner
for file, cost_tar, k_tar in targets:    
    G = read_input('inputs/'+file)
    out_dir = str(Path('outputs') / f"{file[:-len('.in')]}.out")
    in_dir = 'inputs/'+file
    df = pd.read_csv('best_scores.csv')
    r = df.index[df['name'] == file]
    original_c = float(df.loc[r, 'score'])
    
    for k in range(2, 22):
        for v in G.nodes:
            G.nodes[v]['team'] = v % k + 1
        sc = score(G)
        if sc < original_c:
            write(G,'inputs/'+file,str(Path('outputs') / f"{file[:-len('.in')]}.out"), file, score_first,k, True)